<a href="https://colab.research.google.com/github/yewon0325/GraphRAG-with-Neo4j-and-LangChain/blob/main/langchain_RAG_llama3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from bs4 import BeautifulSoup  # Python에서 HTML과 XML 문서를 파싱
import urllib.request
import pandas as pd
import datetime

def busan_Jingu_news(result):
    for i in range(0, 1):
        dataSid = 3907022 - i
        news_url = 'https://www.busanjin.go.kr/board/view.busanjin?boardId=BBS_0000031&menuCd=DOM_000000110003001000&paging=ok&startPage=1&dataSid=%d' % dataSid
        #print(news_url)

        html = urllib.request.urlopen(news_url)
        htmlnew = BeautifulSoup(html, 'html.parser')
        tag_body = htmlnew.find('body')
        class_substan = tag_body.select('.substan')

        paragraphs = class_substan[0].find_all('p')
        strong_texts = [p.text.strip() for p in paragraphs if p.text.strip()]
        return strong_texts

result = []

print(busan_Jingu_news(result))


['부산진구, 2024년 재난대응 안전한국훈련 실시', '부산진구(구청장 김영욱)는 10월 25일 다중밀집시설 지진․화재 복합재난을 가정한 2024 재난대응 안전한국훈련을 토론․현장훈련으로 당감동 소재 한국소재융합연구원에서 실시했다고 밝혔다.', '이날 토론훈련은 단편적 발표방식에서 벗어나 문제 해결형 훈련으로 부산진구의 재난대응 능력을 점검하고 유관기관 간의 협력체계를 강화하는데 중점을 두었다.', '현장훈련은 지진으로 인한 화재 발생 직후 긴급구조통제단과 통합지원본부 운영 등을 통해 초기 대응과 화재 진압, 인명구조, 복구 및 수습 등으로 진행되었다.', '이번 훈련에는 11개 유관기관과 3개 민간단체가 참여하였고 지역에서 가장 빈번한 재난유형인 화재로부터 주민 피해를 최소화에 하는데 역점을 두고 훈련을 실시했다.', '김영욱 구청장은 “앞으로 재난대응 안전한국훈련을 통해 재난으로부터 구민의 인명과 재산 피해를 최소화하는데 만전을 기하겠다”고 밝혔다.']
